# This is a notebook to develop a CNN model for RSO orbit prediction from collision data

## Loading data

In [1]:
import sys
sys.path.append("../modules")
import numpy as np
import pandas as pd
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
import sklearn.metrics as metrics
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
import xgboost as xgb
from sklearn.utils.validation import check_X_y, check_array

from poliastro.bodies import Earth
from poliastro.twobody import Orbit
from poliastro.plotting import *
from poliastro.constants import J2000
from poliastro.util import time_range
from datetime import datetime, timedelta
from astropy import time
from astropy import units as u
import plotly.io as pio
from mlmodels import cnn_model, xgb_model, train_ensamble, ensamble_predict, ensamble_eval
from orbit_module import tle_hist
from utilities import nn_config
pio.renderers.default = "vscode"

data = pd.read_csv('../data/training_data.csv')
test_data = pd.read_csv('../data/test_data.csv')

arct = {'hlayers': 3,
        'ipnodes': 20,
        'nodes': [50, 20, 30 ],
        'activation': ['relu', 'tanh', 'selu']
        }

ann_arct = {'hlayers': 2,
            'ipnodes': 20,
            'nodes': [20, 10],
            'activation': ['tanh', 'tanh', 'tanh']
            }

## Data preprocessing

In [2]:
# Starlink training data
X = data[['x', 'y', 'z', 'vx', 'vy', 'vz', 'delvx', 'delvy', 'delvz']]
y = data[['rso_x', 'rso_y', 'rso_z', 'rso_vx', 'rso_vy', 'rso_vz']].values- \
    data[['x', 'y', 'z', 'vx', 'vy', 'vz']].values
y = pd.DataFrame(y, columns=['delx', 'dely', 'delz', 'delvx', 'delvy', 'delvz'])
y['rso_m'] = data['rso_m']

n_inputs, n_outputs = 6, 1
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=123, test_size=0.01)

# Lemur test data
X_lemur = test_data[['x', 'y', 'z', 'vx', 'vy', 'vz', 'delvx', 'delvy', 'delvz']]
y_lemur = test_data[['rso_x', 'rso_y', 'rso_z', 'rso_vx', 'rso_vy', 'rso_vz']].values- \
    test_data[['x', 'y', 'z', 'vx', 'vy', 'vz']].values
y_lemur = pd.DataFrame(y_lemur, columns=['delx', 'dely', 'delz', 'delvx', 'delvy', 'delvz'])
y_lemur['rso_m'] = test_data['rso_m']

## Training

In [3]:
# parallel NN
model_cnn = cnn_model(ann_arct)
model_cnn.fit(X_train[['x', 'y', 'z', 'delvx', 'delvy', 'delvz']], y_train, n_inputs, n_outputs)

# Ensemble NN
model_p, model_e = train_ensamble(X_train, y_train, n_inputs, n_outputs, arct, method = "cnn")
# fit the model on all data

## Testing

In [4]:
y_hat1 = model_cnn.predict(X_lemur[['x', 'y', 'z', 'delvx', 'delvy', 'delvz']])
y_hat2 = ensamble_predict(model_p, model_e, X_lemur[['x', 'y', 'z', 'delvx', 'delvy', 'delvz']])

frame1 = OrbitPlotter3D()
for case in range(0,4):
    print('Error:', y_hat1[case,:].flatten() - y_test.iloc[case,:].values)
    epoch = time.Time('2021-06-7 12:00:00', format='iso', scale='utc')
    r_hat = np.array(y_hat1[case, 0:3].flatten(), dtype='float') + X_test.iloc[case, 0:3].values
    v_hat = np.array(y_hat1[case, 3:6].flatten(), dtype='float') + X_test.iloc[case, 3:6].values
    y_true = y_test.iloc[case,0:6].values + X_test.iloc[case, 0:6].values
    rso_estimate = Orbit.from_vectors(Earth, r_hat*u.km, v_hat*u.km/u.s, epoch)
    rso_true = Orbit.from_vectors(Earth, y_true[0:3].flatten()*u.km, y_true[3:6].flatten()*u.km/u.s, epoch)

    frame1.plot(rso_estimate, label = 'RSO est')
    frame1.plot(rso_true, label = 'RSO true')
frame1.show()

Error: [ 0.01439413 -0.00102333 -0.04319481  0.1515773  -0.0037172  -0.09090156
  0.0669939 ]
Error: [ 0.10241646 -0.09821551 -0.1555637  -0.02792325 -0.03268845 -0.01717464
 -0.06091167]
Error: [-0.06731693  0.06424891  0.05633732 -0.00327995 -0.12763082  0.03473345
 -0.04801028]
Error: [-0.06086789  0.10951715  0.05737728 -0.07031239  0.15477978  0.02331104
  0.05244762]


In [5]:
eval_nn = model_cnn.eval_models(X_lemur[['x', 'y', 'z', 'delvx', 'delvy', 'delvz']], y_lemur)
eval_nn

,Error,RMSE,R^2,STD
0,delx,0.114535,0.068381,0.122708
1,dely,0.096172,-0.059425,0.099884
2,delz,0.111438,-0.132630,0.108445
3,delvx,0.075386,-0.075763,0.078284
4,delvy,0.074915,-0.064705,0.074948
5,delvz,0.077705,-0.067920,0.077992
6,rso_m,0.246608,-0.011489,0.253029


In [12]:
eval_ens = ensamble_eval(model_p, model_e, X_lemur[['x', 'y', 'z', 'delvx', 'delvy', 'delvz']], y_lemur)
eval_ens

,RMSE,STD
0,0.014885,0.120069
1,0.009374,0.093739
2,0.013273,0.105357
3,0.005236,0.069575
4,0.007840,0.074314
5,0.009913,0.076329
6,0.061224,0.245090


## XGBoost prediction

In [7]:
default_params = {
        'booster': 'dart',
        'tree_method': 'hist'
    }

#parallel
model_xgb = xgb_model(**default_params)
model_xgb.fit(X_train[['x', 'y', 'z', 'delvx', 'delvy', 'delvz']].values, y_train)
# ensamble
model_p_xgb, model_e_xgb = train_ensamble(X_train, y_train, n_inputs, n_outputs, arct, method = "xgb")


In [8]:
y_hat_xgb1 = model_xgb.predict(X_lemur[['x', 'y', 'z', 'delvx', 'delvy', 'delvz']].values)
y_hat_xgb2 = ensamble_predict(model_p_xgb, model_e_xgb, X_lemur[['x', 'y', 'z', 'delvx', 'delvy', 'delvz']])

In [9]:
frame1 = OrbitPlotter3D()
for case in range(0, 4):
    print('Error:', y_hat_xgb2[case,:] - y_test.iloc[case,:].values)
    epoch = time.Time('2021-06-7 12:00:00', format='iso', scale='utc')
    r_hat = np.array(y_hat_xgb2[case, 0:3], dtype='float') + X_test.iloc[case, 0:3].values
    v_hat = np.array(y_hat_xgb2[case, 3:6], dtype='float') + X_test.iloc[case, 3:6].values
    y_true = y_test.iloc[case,0:6].values + X_test.iloc[case, 0:6].values
    rso_estimate = Orbit.from_vectors(Earth, r_hat*u.km, v_hat*u.km/u.s, epoch)
    rso_true = Orbit.from_vectors(Earth, y_true[0:3].flatten()*u.km, y_true[3:6].flatten()*u.km/u.s, epoch)

    frame1.plot(rso_estimate, label = 'RSO est')
    frame1.plot(rso_true, label = 'RSO true')
frame1.show()

Error: [ 0.11362507 -0.0001857  -0.05357191  0.0717947   0.04800072  0.05879229
 -0.06351243]
Error: [ 0.17455153 -0.14141141 -0.17252015 -0.15239716  0.01570626  0.06413466
  0.02217929]
Error: [-0.09933909 -0.00709418  0.09723596 -0.16448056  0.04870856  0.18548214
  0.11571777]
Error: [ 0.12714511  0.13520823 -0.00308163 -0.09896974  0.05392199  0.02061901
 -0.0147593 ]


In [10]:
eval_xgb = model_xgb.eval_models(X_lemur[['x', 'y', 'z', 'delvx', 'delvy', 'delvz']].values, y_lemur)
eval_xgb

,Error,RMSE,R^2,STD
0,delx,0.121618,-0.050396,0.118574
1,dely,0.096194,-0.059925,0.096044
2,delz,0.112970,-0.163990,0.110322
3,delvx,0.091695,-0.591585,0.088590
4,delvy,0.091996,-0.605577,0.091272
5,delvz,0.089994,-0.432406,0.089874
6,rso_m,0.266588,-0.182029,0.266581


In [11]:
eval_ens_xgb = ensamble_eval(model_p_xgb, model_e_xgb, X_lemur[['x', 'y', 'z', 'delvx', 'delvy', 'delvz']], y_lemur)
eval_ens_xgb

,RMSE,STD
0,0.018151,0.121852
1,0.011799,0.108247
2,0.012468,0.110461
3,0.017716,0.086527
4,0.017582,0.104885
5,0.016529,0.085536
6,0.067244,0.259204


In [41]:
test = pd.DataFrame([eval_ens_xgb['RMSE'].values.tolist(), eval_ens['RMSE'].values.tolist()], columns = ['x', 'y', 'z', 'delvx', 'delvy', 'delvz', 'm'])

In [42]:
test

,x,y,z,delvx,delvy,delvz,m
0,0.018151,0.011799,0.012468,0.017716,0.017582,0.016529,0.067244
1,0.014885,0.009374,0.013273,0.005236,0.007840,0.009913,0.061224
